In [79]:
import nltk
from nltk.corpus import treebank 
from nltk.tree import Tree
import csv 
import pandas as pd
import numpy as np
import seaborn as sns
import string
import re
import math
import sklearn
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
!ls "/content/drive/My Drive/Colab Notebooks/data"

adversarial_swap_train.csv  adversarial_swap_train_final.csv


In [81]:
df = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/data/adversarial_swap_train_final.csv")
df.head(10)

,Unnamed: 0,text,sentiment,classification,clean_text,further_clean_text
0,1,Sooo SAD I will miss you here in San Diego!!!,negative,0.0,"['sooo', 'sad', 'miss', 'san', 'diego']","['sooo', 'sad', 'miss', 'san', 'diego']"
1,2,my boss is bullying me...,negative,0.0,"['boss', 'bullying']","['bos', 'bullying']"
2,3,what interview! leave me alone,negative,0.0,"['interview', 'leave', 'alone']","['interview', 'leave', 'alone']"
3,4,"Sons of ****, why couldn`t they put them on t...",negative,0.0,"['sons', 'couldnt', 'put', 'releases', 'alread...","['son', 'couldnt', 'put', 'release', 'already'..."
4,6,2am feedings for the baby are fun when he is a...,positive,1.0,"['feedings', 'baby', 'fun', 'smiles', 'coos']","['feeding', 'baby', 'fun', 'smile', 'coo']"
5,9,Journey!? Wow... u just became cooler. hehe....,positive,1.0,"['journey', 'wow', 'u', 'became', 'cooler', 'h...","['journey', 'wow', 'u', 'became', 'cooler', 'h..."
6,11,I really really like the song Love Story by Ta...,positive,1.0,"['really', 'really', 'like', 'song', 'love', '...","['really', 'really', 'like', 'song', 'love', '..."
7,12,My Sharpie is running DANGERously low on ink,negative,0.0,"['sharpie', 'running', 'dangerously', 'low', '...","['sharpie', 'running', 'dangerously', 'low', '..."
8,13,i want to go to music tonight but i lost my vo...,negative,0.0,"['want', 'go', 'music', 'tonight', 'lost', 'vo...","['want', 'go', 'music', 'tonight', 'lost', 'vo..."
9,15,"Uh oh, I am sunburned",negative,0.0,"['uh', 'oh', 'sunburned']","['uh', 'oh', 'sunburned']"


In [ ]:
!pip3 install transformers
import transformers
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [83]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

In [84]:
def encode_for_bert(data, maximum_length):
    input_ids = []
    attention_masks = []
  
    for i in data:
        sentence = ' '.join(i)
        encoded = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maximum_length, pad_to_max_length=True, return_attention_mask=True, truncation=True)
      
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids),np.array(attention_masks)

In [85]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(df['further_clean_text'], df['classification'], test_size=0.2)
train_input_ids, train_attention_masks = encode_for_bert(X_train_raw, 23)
test_input_ids, test_attention_masks = encode_for_bert(X_test_raw, 23)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [86]:
def add_sparse_layers(bert_model):
    input_ids = tf.keras.Input(shape=(23,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(23,),dtype='int32')
  
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer='l1')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [89]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-large-uncased')

ResourceExhaustedError: ignored

In [77]:
model = add_sparse_layers(bert_model)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 23)]         0           []                               
                                                                                                  
 input_18 (InputLayer)          [(None, 23)]         0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  multiple             335141888   ['input_17[0][0]',               
                                                                  'input_18[0][0]']               
                                                                                                  
 dense_10 (Dense)               (None, 32)           32800       ['tf_bert_model_1[3][1]']  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [78]:
history = model.fit([train_input_ids,train_attention_masks], y_train_raw, validation_split=0.2, epochs=2,batch_size=10)

Epoch 1/2


ResourceExhaustedError: ignored

In [61]:
result = model.predict([test_input_ids,test_attention_masks])
result = np.round(result).astype(int)

In [62]:
from sklearn.metrics import accuracy_score, f1_score

In [63]:
print(accuracy_score(y_test_raw, result))
print(f1_score(y_test_raw, result))

0.5030562347188264
0.6693777958519723
